In [2]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle
import joblib
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [12]:
loan_df = pd.read_csv('./data/loan_result.csv')

In [7]:
merged_ver_2_0 = pd.read_csv('./valiData/merged_ver_2_0.csv',index_col=0)

# 의미를 부여하여 결측치 처리하기

## 데이터 확인

In [8]:
merged_ver_2_0.shape

(13527250, 23)

In [9]:
(merged_ver_2_0.isnull().sum())[merged_ver_2_0.isnull().sum()!=0]

birth_year                               108227
gender                                   108227
credit_score                            1311773
company_enter_month                      306537
personal_rehabilitation_yn              2934375
personal_rehabilitation_complete_yn    11162940
existing_loan_cnt                       2685709
existing_loan_amt                       3890163
loan_limit                                   10
loan_rate                                    10
is_applied                              3257239
dtype: int64

In [10]:
(merged_ver_2_0[merged_ver_2_0['is_applied'].isnull()==True].isnull().sum())[merged_ver_2_0[merged_ver_2_0['is_applied'].isnull()==True].isnull().sum()!=0]

birth_year                               30960
gender                                   30960
credit_score                            256483
company_enter_month                      75881
personal_rehabilitation_yn               14664
personal_rehabilitation_complete_yn    2538395
existing_loan_cnt                       541898
existing_loan_amt                       846023
loan_limit                                   2
loan_rate                                    2
is_applied                             3257239
dtype: int64

In [13]:
len(loan_df[loan_df['is_applied'].isnull()==True])

3257239

## existing_loan_cnt & existing_loan_amt

In [15]:
merged_ver_2_0[(merged_ver_2_0['existing_loan_amt'].isnull()==True)]['existing_loan_cnt'].value_counts()

1.0    1204454
Name: existing_loan_cnt, dtype: int64

In [16]:
merged_ver_2_0.loc[merged_ver_2_0[(merged_ver_2_0['existing_loan_amt'].isnull()==True) & (merged_ver_2_0['existing_loan_cnt']==1)].index, 'existing_loan_amt'] = -100

In [17]:
(merged_ver_2_0.isnull().sum())[merged_ver_2_0.isnull().sum()!=0]

birth_year                               108227
gender                                   108227
credit_score                            1311773
company_enter_month                      306537
personal_rehabilitation_yn              2934375
personal_rehabilitation_complete_yn    11162940
existing_loan_cnt                       2685709
existing_loan_amt                       2685709
loan_limit                                   10
loan_rate                                    10
is_applied                              3257239
dtype: int64

In [18]:
set(merged_ver_2_0[merged_ver_2_0['existing_loan_amt'].isnull()==True].index) ==\
set(merged_ver_2_0[merged_ver_2_0['existing_loan_cnt'].isnull()==True].index)

True

In [19]:
merged_ver_2_0.loc[merged_ver_2_0[merged_ver_2_0['existing_loan_amt'].isnull()==True].index, ['existing_loan_amt', 'existing_loan_cnt']]=0

In [20]:
(merged_ver_2_0.isnull().sum())[merged_ver_2_0.isnull().sum()!=0]

birth_year                               108227
gender                                   108227
credit_score                            1311773
company_enter_month                      306537
personal_rehabilitation_yn              2934375
personal_rehabilitation_complete_yn    11162940
loan_limit                                   10
loan_rate                                    10
is_applied                              3257239
dtype: int64

## personal_rehabilitation_yn & personal_rehabilitation_complete_yn

In [21]:
merged_ver_2_0[(merged_ver_2_0['personal_rehabilitation_complete_yn'].isnull()==True)]['personal_rehabilitation_yn'].value_counts()

0.0    8228565
Name: personal_rehabilitation_yn, dtype: int64

In [24]:
merged_ver_2_0.loc[merged_ver_2_0[(merged_ver_2_0['personal_rehabilitation_yn']==0)].index, 'personal_rehabilitation_complete_yn'] = -1

In [25]:
set(merged_ver_2_0[merged_ver_2_0['personal_rehabilitation_yn'].isnull()==True].index)\
== set(merged_ver_2_0[merged_ver_2_0['personal_rehabilitation_complete_yn'].isnull()==True].index)

True

In [26]:
merged_ver_2_0.loc[merged_ver_2_0[merged_ver_2_0['personal_rehabilitation_yn'].isnull()==True].index, 'personal_rehabilitation_yn']=0

In [27]:
merged_ver_2_0.loc[merged_ver_2_0[merged_ver_2_0['personal_rehabilitation_complete_yn'].isnull()==True].index, 'personal_rehabilitation_complete_yn']=-1

In [28]:
(merged_ver_2_0.isnull().sum())[merged_ver_2_0.isnull().sum()!=0] # personal_rehabilitation_complete_yn  잘못표기됨

birth_year              108227
gender                  108227
credit_score           1311773
company_enter_month     306537
loan_limit                  10
loan_rate                   10
is_applied             3257239
dtype: int64

## company_enter_month     

In [29]:
merged_ver_2_0['company_enter_month']

0           202201.0
1           202201.0
2           202201.0
3           202201.0
4           202201.0
              ...   
13527245    202204.0
13527246    202204.0
13527247    202204.0
13527248    202204.0
13527249    202204.0
Name: company_enter_month, Length: 13527250, dtype: float64

In [30]:
merged_ver_2_0['income_type'].unique()

array(['EARNEDINCOME', 'OTHERINCOME', 'EARNEDINCOME2', 'FREELANCER',
       'PRIVATEBUSINESS', 'PRACTITIONER'], dtype=object)

In [31]:
for income_type in merged_ver_2_0['income_type'].unique():
    print(income_type, merged_ver_2_0[merged_ver_2_0['income_type']==income_type]['company_enter_month'].isnull().sum())

EARNEDINCOME 55
OTHERINCOME 306459
EARNEDINCOME2 0
FREELANCER 0
PRIVATEBUSINESS 23
PRACTITIONER 0


In [32]:
for income_type in merged_ver_2_0[merged_ver_2_0['is_applied'].isnull()==True]['income_type'].unique():
    print(income_type, merged_ver_2_0[(merged_ver_2_0['is_applied'].isnull()==True) & (merged_ver_2_0['income_type']==income_type)]\
          ['company_enter_month'].isnull().sum())

EARNEDINCOME 23
FREELANCER 0
OTHERINCOME 75858
PRACTITIONER 0
EARNEDINCOME2 0
PRIVATEBUSINESS 0


In [33]:
merged_ver_2_0[merged_ver_2_0['income_type']=='OTHERINCOME']['company_enter_month'].value_counts()

202201.0      6119
202203.0      6017
202204.0      5121
202205.0      4153
202202.0      3403
              ... 
20210620.0       1
20180611.0       1
20180623.0       1
20120328.0       1
201010.0         1
Name: company_enter_month, Length: 833, dtype: int64

In [34]:
merged_ver_2_0.loc[merged_ver_2_0[merged_ver_2_0['company_enter_month']<1_00_00_00].index, 'company_enter_month']= merged_ver_2_0[merged_ver_2_0['company_enter_month']<1_00_00_00]['company_enter_month'].copy()*100+1

In [35]:
merged_ver_2_0.columns

Index(['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'loanapply_insert_time',
       'bank_id', 'product_id', 'loan_limit', 'loan_rate', 'is_applied'],
      dtype='object')

In [36]:
merged_ver_2_0['loanapply_insert_day']=pd.to_datetime(merged_ver_2_0['loanapply_insert_time'])

In [38]:
merged_ver_2_0['company_enter_month']=pd.to_datetime(merged_ver_2_0['company_enter_month'], format='%Y%m%d')

In [39]:
merged_ver_2_0['work_days']=(merged_ver_2_0['loanapply_insert_day']-merged_ver_2_0['company_enter_month']).dt.days

In [40]:
merged_ver_2_0.drop(['loanapply_insert_day'],inplace=True,axis=1)

In [41]:
merged_ver_2_0.drop(['company_enter_month'],inplace=True,axis=1)

In [42]:
merged_ver_2_0.loc[merged_ver_2_0[merged_ver_2_0['work_days'].isnull()==True].index, 'work_days']=0

In [43]:
(merged_ver_2_0.isnull().sum())[merged_ver_2_0.isnull().sum()!=0]

birth_year       108227
gender           108227
credit_score    1311773
loan_limit           10
loan_rate            10
is_applied      3257239
dtype: int64

In [45]:
(merged_ver_2_0[merged_ver_2_0['is_applied'].isnull()==True].isnull().sum())[merged_ver_2_0[merged_ver_2_0['is_applied'].isnull()==True].isnull().sum()!=0]

birth_year        30960
gender            30960
credit_score     256483
loan_limit            2
loan_rate             2
is_applied      3257239
dtype: int64

In [44]:
merged_ver_2_0.to_csv('./valiData/merged_ver_3_0.csv')

# 버전업

## merged_ver_3_1

In [2]:
merged_ver_3_0=pd.read_csv('./valiData/merged_ver_3_0.csv',index_col=0)

In [3]:
merged_df = pd.read_csv('./customData/merged_user_loan.csv',index_col=0)

In [4]:
merged_ver_3_0.shape, merged_df.shape, 

((13527250, 23), (13527250, 23))

In [5]:
merged_ver_3_0['loan_limit'] = merged_df['loan_limit']
merged_ver_3_0['loan_rate'] = merged_df['loan_rate']

In [6]:
merged_ver_3_0.to_csv('./expermData/merged_ver_3_1.csv')

In [7]:
merged_ver_3_0.isnull().sum()

application_id                               0
user_id                                      0
birth_year                              108227
gender                                  108227
insert_time                                  0
credit_score                           1311773
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                7382
loan_rate                                 7382
is_applied   

## merged_ver_3_2

In [3]:
merged_ver_3_1=pd.read_csv('./expermData/merged_ver_3_1.csv',index_col=0)

In [4]:
merged_ver_3_1['loan_limit'].fillna(0, inplace=True)
merged_ver_3_1['loan_rate'].fillna(0, inplace=True)

In [5]:
merged_ver_3_1.isnull().sum()

application_id                               0
user_id                                      0
birth_year                              108227
gender                                  108227
insert_time                                  0
credit_score                           1311773
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied   

In [6]:
merged_ver_3_1.to_csv('./expermData/merged_ver_3_2.csv')

## merged_ver_3_3

In [7]:
merged_ver_3_1.replace('LIVING', '생활비', inplace=True)
merged_ver_3_1.replace('BUSINESS','사업자금', inplace=True)
merged_ver_3_1.replace('SWITCHLOAN','대환대출', inplace=True)
merged_ver_3_1.replace('ETC','기타', inplace=True)
merged_ver_3_1.replace('BUYCAR','자동차구입', inplace=True)
merged_ver_3_1.replace('INVEST','투자', inplace=True)
merged_ver_3_1.replace('HOUSEDEPOSIT','전월세보증금', inplace=True)
merged_ver_3_1.replace('BUYHOUSE','주택구입', inplace=True)

In [8]:
merged_ver_3_1.to_csv('./expermData/merged_ver_3_3.csv')

## merged_ver_4

In [3]:
merged_ver_3_3 = pd.read_csv('./expermData/merged_ver_3_3.csv')

In [4]:
merged_ver_3_3['gender'].replace('0', 'F', inplace=True)
merged_ver_3_3['gender'].replace('1', 'M', inplace=True)
merged_ver_3_3['gender'].fillna('A', inplace=True)

In [6]:
merged_ver_3_3['birth_year'].fillna(merged_ver_3_3['birth_year'].min(),\
inplace=True)

In [7]:
merged_ver_3_3.to_csv('./expermData/merged_ver_4_0.csv')